In [ ]:
import mne
import numpy as np
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.svm import SVC 
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA 
from mne.preprocessing import ICA
from mne.decoding import CSP
import matplotlib.pyplot as plt 
import numpy as np
from mne.decoding import CSP

In [ ]:
#Data Preprocessing Enhancement 
raw = mne.io.read_raw_gdf('data_gdf/A01T.gdf',eog=['EOG-left', 'EOG-central', 'EOG-right'],preload=True)

def preprocessing_eeg(raw):
    """
    Preprocessing raw data by applying a band-pass filter and ICA.
    Parameter: raw(mne.io.Raw): Raw EEG Data.
    Returns: raw_clean(mne.io.Raw):Cleaned EEG data after artifact removal.
    """
    #Apply band-pass filter
    raw.filter(8.,30.,fir_design='firwin',skip_by_annotation='edge')

    #Apply ICA for artifact removal
    ica = ICA(n_components=15,random_state=97,max_iter=800)
    ica.fit(raw)
    ica.exclude = [0] #indices of components to exclude (such as eye blinks)
    raw_clean = ica.apply(raw)

    return raw_clean

# to run the function individually and check the preprocessing:
#preprocessing_eeg(raw)

In [ ]:
# Feature Extraction using CSP and Time-Frequency Analysis

def motor_img_reading(path):
    raw = mne.io.read_raw_gdf(path,eog=['EOG-left', 'EOG-central', 'EOG-right'],preload=True)
    raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])
    raw.set_eeg_reference()
    events = mne.events_from_annotations(raw)
    epochs = mne.Epochs(raw,events[0],event_id=[7,8,9,10],tmin=-0.1,tmax=0.7,on_missing='Alert')
    labels=epochs.events[:,-1] 
    features=epochs.get_data()
    return labels,features

labels,features = motor_img_reading('data_gdf/A01T.gdf')
events = mne.events_from_annotations(raw)
epochs = mne.Epochs(raw,events[0],event_id=[7,8,9,10],tmin=-0.1,tmax=0.7,on_missing='Alert')

In [ ]:
def extract_features(epochs):
    """
    Extracts features using CSP and calculates power spectral density.

    Parameters:
    epochs (mne.Epochs): Epochs object containing the EEG data.

    Returns:
    combined_features (np.array): Combined CSP and PSD features.
    """
    # Initialize CSP with fewer components and regularization
    csp = CSP(n_components=2, reg='ledoit_wolf', log=True, cov_est='epoch')
    
    # Apply CSP on epochs data
    try:
        features = csp.fit_transform(epochs.get_data(), epochs.events[:, -1])
    except np.linalg.LinAlgError as e:
        print(f"Error in CSP transformation: {e}")
        return None

    # Calculate PSD for each epoch
    psd_features = np.array([mne.time_frequency.psd_multitaper(epoch, fmin=8., fmax=30., verbose=False)[0] for epoch in epochs])
    
    # Flatten the PSD features
    psd_features = psd_features.reshape(psd_features.shape[0], -1)
    
    # Combine CSP features with PSD features
    combined_features = np.hstack((features, psd_features))
    
    return combined_features
"""
# Example of calling the function
combined_features = extract_features(epochs)
if combined_features is not None:
    print(f"Extracted features shape: {combined_features.shape}")
else:
    print("Feature extraction failed due to CSP convergence issue.")
"""

In [ ]:
# Model Building and Evaluation
def train_model(features, labels):
    """
    Trains and evaluates a classifier using cross-validation.

    Parameters:
    features (np.array): Extracted features.
    labels (np.array): Corresponding labels.

    Returns:
    clf (object): Trained classifier.
    """
    # Initialize the classifier (e.g., SVM with LDA)
    clf = Pipeline([('lda', LDA()), ('svm', SVC(kernel='linear', probability=True))])
    
    # Perform cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(clf, features, labels, cv=cv, scoring='accuracy')
    
    print(f'Cross-validation accuracy: {np.mean(scores):.2f} ± {np.std(scores):.2f}')
    
    # Train the classifier on the entire dataset
    clf.fit(features, labels)
    
    return clf

In [ ]:
# Plotting Enhancements
def plot_topomap(epochs):
    """
    Plots the topographical map of EEG data.

    Parameters:
    epochs (mne.Epochs): Epochs object containing the EEG data.
    """
    # Calculate the average evoked response
    evoked = epochs.average()
    
    # Plot the topomap
    evoked.plot_topomap(times=np.linspace(0, 0.5, 5), ch_type='eeg', show=True)

def motor_img_analysis(path):
    """
    Complete workflow for motor imagery EEG analysis including preprocessing, feature extraction, model training, and visualization.

    Parameters:
    path (str): Path to the GDF file.
    """
    # Load and preprocess the data
    raw = mne.io.read_raw_gdf(path, eog=['EOG-left', 'EOG-central', 'EOG-right'], preload=True)
    raw_clean = preprocessing_eeg(raw)
    
    # Extract events and epochs
    events = mne.events_from_annotations(raw_clean)
    epochs = mne.Epochs(raw_clean, events[0], event_id=[7, 8, 9, 10], tmin=-0.1, tmax=0.7, on_missing='alert')
    labels = epochs.events[:, -1]
    
    # Extract features
    features = extract_features(epochs)
    
    # Train and evaluate the model
    clf = train_model(features, labels)
    
    # Plot topomap to visualize the spatial distribution of EEG activity
    plot_topomap(epochs)
    
    return clf

# Run the analysis
#model = motor_img_analysis('data_gdf/A01T.gdf')

In [ ]:
import mne
import numpy as np
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from mne.decoding import CSP
from mne import Epochs

def preprocess_and_extract_features(path):
    """
    Reads the EEG data, preprocesses it, and extracts features using CSP and PSD.

    Parameters:
    path (str): Path to the GDF file.

    Returns:
    features (np.array): Extracted features.
    labels (np.array): Corresponding labels.
    """
    # Read the raw GDF file
    raw = mne.io.read_raw_gdf(path, eog=['EOG-left', 'EOG-central', 'EOG-right'], preload=True)
    raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])
    raw.set_eeg_reference()
    
    # Apply a band-pass filter
    raw.filter(8., 30., fir_design='firwin', skip_by_annotation='edge')
    
    # Extract events and epochs
    events = mne.events_from_annotations(raw)
    epochs = mne.Epochs(raw, events[0], event_id=[7, 8, 9, 10], tmin=-0.1, tmax=0.7, on_missing='alert')
    
    labels = epochs.events[:, -1]
    
    # Initialize CSP with regularization
    csp = CSP(n_components=3, reg='ledoit_wolf', log=True, cov_est='epoch')
    
    # Apply CSP on epochs data
    try:
        features = csp.fit_transform(epochs.get_data(), labels)
    except np.linalg.LinAlgError as e:
        print(f"SVD did not converge: {e}")
        return None, None
    
    # Calculate PSD for each epoch
    psd_features = np.array([mne.time_frequency.psd_multitaper(epoch, fmin=8., fmax=30., verbose=False)[0] for epoch in epochs])
    
    # Flatten the PSD features
    psd_features = psd_features.reshape(psd_features.shape[0], -1)
    
    # Combine CSP features with PSD features
    combined_features = np.hstack((features, psd_features))
    
    return combined_features, labels

def train_svm(features, labels):
    """
    Trains an SVM classifier on the provided features.

    Parameters:
    features (np.array): Extracted features.
    labels (np.array): Corresponding labels.

    Returns:
    clf (object): Trained classifier.
    """
    if features is None or labels is None:
        print("No features to train on due to previous errors.")
        return None
    
    # Standardize features
    scaler = StandardScaler()
    
    # Initialize the classifier with a linear kernel and more iterations
    svm = SVC(kernel='linear', C=1.0, max_iter=10000)
    
    # Create a pipeline
    clf = Pipeline([('scaler', scaler), ('svm', svm)])
    
    # Train the classifier
    clf.fit(features, labels)
    
    return clf

# Set the path to the specific data file
path = 'data_gdf/A01T.gdf'

# Preprocess data and extract features
#features, labels = preprocess_and_extract_features(path)

# Train the SVM model
#if features is not None:
 #   clf = train_svm(features, labels)